In [136]:
import numpy as np
from collections import defaultdict
import csv

In [16]:
dat = np.load('data/consolidations/yak_cons_2015-05-27 14:01:41.586356.npy')

# dat[i, 0] id
# dat[i, 1] uni
# dat[i, 2] handle
# dat[i, 3] yak
# dat[i, 4] votes

[u'R/5563fb102e54cacef0463fb1f5f54' u'princeton' u''
 u'Is Perez crazy. Ancellotti is our last best hope' u'6.0']


In [76]:
# Warriner et. al. affect score import
lines = [line.rstrip('\n').split(",") for line in open('Ratings_Warriner_et_al.csv')]
WARRINER_AFFECT = defaultdict(float)
for line in lines[1:]:
    WARRINER_AFFECT[line[1].lower()] = float(line[2])

In [140]:
# Feature functions

'''
Pointedness
'''
# Capitalization counter
# https://github.com/MathieuCliche/Sarcasm_detector
def cap_feature(yak):
    counter = 0
    thresh = 4 # for alternative feature below
    for j in range(len(yak[2])):
        features['Capitalization'] += int(yak[2][j].isupper())
#     features['Capitalization'] = int(counter >= thresh)
    return features
    
# Punctuation
punc_marks = ['.', '...', ';', ':', '?', '!', '\'', '\"']

def punc_feature(yak):
    features = defaultdict(float)
    for i in punc_marks:
        features[('Punctuation ' + i)] += float(i in yak[2])
    return features

'''
Delta affect and delta sentiment
'''

def affect_feature(yak):
    features = defaultdict(float)
    words = yak[2].split(" ")
    scores = np.zeros((len(words)))
    for ind, word in enumerate(words):
        scores[ind] = WARRINER_AFFECT[word.lower()]
    features["Delta Affect"] = (np.amax(scores) - np.amin(scores))
    return features
    
def sentiment_feature(yak):
    features = defaultdict(float)
    features["Delta Sentiment"] = 0.0
    return features

def interrogative_feature(yak):
    features = defaultdict(float)
    model_words = ['what', 'where', 'when', 'why', 'who']
    auxiliary_verbs = ['am', 'is', 'are', 'was', 'were', 'am', 'do', 'did', 'does']
    words = yak[2].split(" ")
    
    is_interrogative = ((words[0] in model_words) and 
                        (words[1] in auxiliary_verbs) and ('?' in yak[2]))
    features["Interrogative"] = float(is_interrogative)
    return features
        

In [159]:
interrogative_feature(dat[0])
dat[19]

array([u'R/5564f10d95002a266c585c9841ac0', u'mit', u'',
       u'People on the internet man, vicious. ', u'7.0'], 
      dtype='<U200')

In [ ]:
# "section leader" and "section leaders"
# punctuation cutting off yaks.
def bigram_feature(yak):
    words = yak[2].lower().split()
    school = yak[0]
    for ind in xrange(len(words) - 1):
        features[(school, words[ind], words[ind + 1])] += 1.0
    return features